# Advanced GigE Camera Configuration

This notebook provides advanced testing and configuration for GigE cameras.

## Features:
- Network interface detection
- GigE camera discovery
- Camera parameter configuration
- Performance testing
- Dual camera synchronization testing

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import subprocess
import socket
import threading
import time
from datetime import datetime
import os
from pathlib import Path
import json

print("✅ Imports successful!")

## Network Interface Detection

In [ ]:
def get_network_interfaces():
    """Get network interface information"""
    try:
        result = subprocess.run(['ip', 'addr', 'show'], capture_output=True, text=True)
        print("🌐 Network Interfaces:")
        print(result.stdout)
        
        # Also check for GigE specific interfaces
        result2 = subprocess.run(['ifconfig'], capture_output=True, text=True)
        if result2.returncode == 0:
            print("\n📡 Interface Configuration:")
            print(result2.stdout)
    except Exception as e:
        print(f"❌ Error getting network info: {e}")

get_network_interfaces()

## GigE Camera Discovery

In [ ]:
def discover_gige_cameras():
    """Attempt to discover GigE cameras on the network"""
    print("🔍 Discovering GigE cameras...")
    
    # Try different methods to find GigE cameras
    methods = [
        "OpenCV with different backends",
        "Network scanning",
        "GStreamer pipeline testing"
    ]
    
    print("\n1. Testing OpenCV backends:")
    backends = [
        (cv2.CAP_GSTREAMER, "GStreamer"),
        (cv2.CAP_V4L2, "V4L2"),
        (cv2.CAP_FFMPEG, "FFmpeg"),
        (cv2.CAP_ANY, "Default")
    ]
    
    for backend_id, backend_name in backends:
        print(f"  Testing {backend_name}...")
        for cam_id in range(5):
            try:
                cap = cv2.VideoCapture(cam_id, backend_id)
                if cap.isOpened():
                    ret, frame = cap.read()
                    if ret:
                        print(f"    ✅ Camera {cam_id} accessible via {backend_name}")
                        print(f"       Resolution: {frame.shape[1]}x{frame.shape[0]}")
                cap.release()
            except Exception as e:
                pass
    
    print("\n2. Testing GStreamer pipelines:")
    # Common GigE camera GStreamer pipelines
    gstreamer_pipelines = [
        "v4l2src device=/dev/video0 ! videoconvert ! appsink",
        "v4l2src device=/dev/video1 ! videoconvert ! appsink",
        "tcambin ! videoconvert ! appsink",  # For TIS cameras
        "aravis ! videoconvert ! appsink",   # For Aravis-supported cameras
    ]
    
    for pipeline in gstreamer_pipelines:
        try:
            print(f"  Testing: {pipeline}")
            cap = cv2.VideoCapture(pipeline, cv2.CAP_GSTREAMER)
            if cap.isOpened():
                ret, frame = cap.read()
                if ret:
                    print(f"    ✅ Pipeline works! Frame shape: {frame.shape}")
                else:
                    print(f"    ⚠️ Pipeline opened but no frames")
            else:
                print(f"    ❌ Pipeline failed")
            cap.release()
        except Exception as e:
            print(f"    ❌ Error: {e}")

discover_gige_cameras()

## Camera Parameter Configuration

In [ ]:
def configure_camera_parameters(camera_id, backend=cv2.CAP_ANY):
    """Configure and test camera parameters"""
    print(f"⚙️ Configuring camera {camera_id}...")
    
    cap = cv2.VideoCapture(camera_id, backend)
    if not cap.isOpened():
        print(f"❌ Cannot open camera {camera_id}")
        return None
    
    # Get current parameters
    current_params = {
        'width': cap.get(cv2.CAP_PROP_FRAME_WIDTH),
        'height': cap.get(cv2.CAP_PROP_FRAME_HEIGHT),
        'fps': cap.get(cv2.CAP_PROP_FPS),
        'brightness': cap.get(cv2.CAP_PROP_BRIGHTNESS),
        'contrast': cap.get(cv2.CAP_PROP_CONTRAST),
        'saturation': cap.get(cv2.CAP_PROP_SATURATION),
        'hue': cap.get(cv2.CAP_PROP_HUE),
        'gain': cap.get(cv2.CAP_PROP_GAIN),
        'exposure': cap.get(cv2.CAP_PROP_EXPOSURE),
        'auto_exposure': cap.get(cv2.CAP_PROP_AUTO_EXPOSURE),
        'white_balance': cap.get(cv2.CAP_PROP_WHITE_BALANCE_BLUE_U),
    }
    
    print("📊 Current Camera Parameters:")
    for param, value in current_params.items():
        print(f"  {param}: {value}")
    
    # Test setting some parameters
    print("\n🔧 Testing parameter changes:")
    
    # Try to set resolution (common GigE resolutions)
    test_resolutions = [(1920, 1080), (1280, 720), (640, 480)]
    for width, height in test_resolutions:
        if cap.set(cv2.CAP_PROP_FRAME_WIDTH, width) and cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height):
            actual_width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
            actual_height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
            print(f"  Resolution {width}x{height}: Set to {actual_width}x{actual_height}")
            break
    
    # Test FPS settings
    for fps in [30, 60, 120]:
        if cap.set(cv2.CAP_PROP_FPS, fps):
            actual_fps = cap.get(cv2.CAP_PROP_FPS)
            print(f"  FPS {fps}: Set to {actual_fps}")
            break
    
    # Capture test frame with new settings
    ret, frame = cap.read()
    if ret:
        print(f"\n✅ Test frame captured: {frame.shape}")
        
        # Display frame
        plt.figure(figsize=(10, 6))
        if len(frame.shape) == 3:
            plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        else:
            plt.imshow(frame, cmap='gray')
        plt.title(f"Camera {camera_id} - Configured")
        plt.axis('off')
        plt.show()
        
        # Save configuration and test image
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        # Save image
        img_path = f"/storage/camera{camera_id}/configured_test_{timestamp}.jpg"
        cv2.imwrite(img_path, frame)
        print(f"💾 Test image saved: {img_path}")
        
        # Save configuration
        config_path = f"/storage/camera{camera_id}/config_{timestamp}.json"
        with open(config_path, 'w') as f:
            json.dump(current_params, f, indent=2)
        print(f"💾 Configuration saved: {config_path}")
    
    cap.release()
    return current_params

# Test configuration (change camera_id as needed)
camera_to_configure = 0
config = configure_camera_parameters(camera_to_configure)

## Dual Camera Testing

In [ ]:
def test_dual_cameras(camera1_id=0, camera2_id=1, duration=5):
    """Test simultaneous capture from two cameras"""
    print(f"📷📷 Testing dual camera capture (cameras {camera1_id} and {camera2_id})...")
    
    # Open both cameras
    cap1 = cv2.VideoCapture(camera1_id)
    cap2 = cv2.VideoCapture(camera2_id)
    
    if not cap1.isOpened():
        print(f"❌ Cannot open camera {camera1_id}")
        return
    
    if not cap2.isOpened():
        print(f"❌ Cannot open camera {camera2_id}")
        cap1.release()
        return
    
    print("✅ Both cameras opened successfully")
    
    # Capture test frames
    ret1, frame1 = cap1.read()
    ret2, frame2 = cap2.read()
    
    if ret1 and ret2:
        print(f"📊 Camera {camera1_id}: {frame1.shape}")
        print(f"📊 Camera {camera2_id}: {frame2.shape}")
        
        # Display both frames side by side
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
        
        if len(frame1.shape) == 3:
            ax1.imshow(cv2.cvtColor(frame1, cv2.COLOR_BGR2RGB))
        else:
            ax1.imshow(frame1, cmap='gray')
        ax1.set_title(f"Camera {camera1_id}")
        ax1.axis('off')
        
        if len(frame2.shape) == 3:
            ax2.imshow(cv2.cvtColor(frame2, cv2.COLOR_BGR2RGB))
        else:
            ax2.imshow(frame2, cmap='gray')
        ax2.set_title(f"Camera {camera2_id}")
        ax2.axis('off')
        
        plt.tight_layout()
        plt.show()
        
        # Save test images
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        cv2.imwrite(f"/storage/camera1/dual_test_{timestamp}.jpg", frame1)
        cv2.imwrite(f"/storage/camera2/dual_test_{timestamp}.jpg", frame2)
        print(f"💾 Dual camera test images saved with timestamp {timestamp}")
        
    else:
        print("❌ Failed to capture from one or both cameras")
    
    # Test synchronized recording
    print(f"\n🎥 Testing synchronized recording for {duration} seconds...")
    
    # Setup video writers
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = 30
    
    if ret1:
        h1, w1 = frame1.shape[:2]
        out1 = cv2.VideoWriter(f"/storage/camera1/sync_test_{timestamp}.mp4", fourcc, fps, (w1, h1))
    
    if ret2:
        h2, w2 = frame2.shape[:2]
        out2 = cv2.VideoWriter(f"/storage/camera2/sync_test_{timestamp}.mp4", fourcc, fps, (w2, h2))
    
    # Record synchronized video
    start_time = time.time()
    frame_count = 0
    
    while time.time() - start_time < duration:
        ret1, frame1 = cap1.read()
        ret2, frame2 = cap2.read()
        
        if ret1 and ret2:
            out1.write(frame1)
            out2.write(frame2)
            frame_count += 1
        else:
            print(f"⚠️ Frame drop at frame {frame_count}")
    
    # Cleanup
    cap1.release()
    cap2.release()
    if 'out1' in locals():
        out1.release()
    if 'out2' in locals():
        out2.release()
    
    elapsed = time.time() - start_time
    actual_fps = frame_count / elapsed
    
    print(f"✅ Synchronized recording complete")
    print(f"📊 Recorded {frame_count} frames in {elapsed:.2f}s")
    print(f"📊 Actual FPS: {actual_fps:.2f}")
    print(f"💾 Videos saved with timestamp {timestamp}")

# Test dual cameras (adjust camera IDs as needed)
test_dual_cameras(0, 1, duration=3)